In [ ]:
import itertools
import importlib

import numpy as np
import networkx as nx

import torch
from torch_geometric import utils as gutils

from awe import features, graph_utils
from awe.data import swde, dataset

for module in [features, graph_utils, dataset, swde]:
    importlib.reload(module)

In [ ]:
sds = swde.Dataset(suffix='-exact')
ds = dataset.DatasetCollection()

In [ ]:
ds.features = [
    features.Depth(),
    features.IsLeaf(),
    features.CharCategories(),
    features.FontSize(),
    features.CharIdentifiers(),
    features.WordIdentifiers()
]

In [ ]:
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting
import bokeh.transform
bokeh.io.output_notebook()

In [ ]:
page = sds.verticals[0].websites[3].pages[188]
page_ctx = ds.prepare_page_context(page)
page.file_path

In [ ]:
[n for n in page_ctx.nodes if n.labels == ['price']][0]

In [ ]:
graph = graph_utils.to_networkx(page_ctx)
graph.nodes[440]

In [ ]:
bokeh_graph = bokeh.plotting.from_networkx(graph, nx.spring_layout)

In [ ]:
label_cmap = bokeh.transform.factor_cmap(
    'label', bokeh.palettes.Spectral5, page.fields)
bokeh_graph.node_renderer.glyph = bokeh.models.Circle(
    size='display_size',
    fill_color=label_cmap
)

In [ ]:
TOOLTIPS = [
    ('index', '$index'),
    ('size', '@display_size'),
    ('id', '@id'),
    ('tag', '@tag'),
    ('text', '@text'),
    ('label', '@label')
]

plot = bokeh.plotting.figure(
    tooltips=TOOLTIPS,
    active_scroll='wheel_zoom'
)

plot.renderers.append(bokeh_graph)
bokeh.io.show(plot)